In [8]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table
import pandas as pd
import sys
import os
import configparser
import copy

# Add the root project directory to Python path
root_path = os.path.abspath('../../')
sys.path.insert(0, root_path)

# Now import from modules
from modules.simulation import UniverseSimulator

plt.style.use('default')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

# DES Y1 Mass-Richness Relation Parameter Calibration

This notebook calibrates Capish mass-richness relation parameters against DES Y1 cluster data.
- Survey area: 1603.5 deg²
- Excludes highest richness bin (>60) for calibration

## Adjustable Parameters

**Modify these values to iterate on the mass-richness relation:**

In [9]:
# Mass-richness relation parameters for iteration
MASS_RICHNESS_PARAMS = {
    # Pivot points
    'pivot_obs_z0': 0.35,         # Pivot redshift
    'pivot_obs_log10m0': 14.3,    # Pivot mass (log10 M_sun)
    
    # Richness scaling
    'params_mean_obs_mu0': 3.2,        # Mean log(richness) at pivot
    'params_mean_obs_muz': 0.0,        # Redshift evolution
    'params_mean_obs_mulog10m': 1.0,   # Mass scaling
    
    # Richness scatter
    'params_stdd_obs_mu0': 0.25,       # Log(richness) scatter
    'params_stdd_obs_muz': 0.0,        # Scatter redshift evolution
    'params_stdd_obs_mulog10m': 0.0,   # Scatter mass evolution
    
    # WL mass relation
    'params_mean_log10mWL_aWL': 1.0,   # WL mass scaling
    'params_mean_log10mWL_bWL': 0.0,   # WL mass offset
    'params_stdd_log10mWLgal': 0.2,    # WL mass scatter (galaxy)
    'params_stdd_log10mWLint': 0.2,    # WL mass scatter (intrinsic)
    'rho_obs_mWL': 0.2                 # Richness-WL mass correlation
}

# Cosmological parameters (Planck 2018)
COSMO_PARAMS = {
    'Omega_m': 0.315,
    'sigma8': 0.811, 
    'Omega_b': 0.049,
    'h': 0.674,
    'ns': 0.965,
    'w0': -1,
    'wa': 0
}

# Survey parameters
DES_SURVEY_AREA_DEG2 = 1603.5
DES_SURVEY_AREA_SR = DES_SURVEY_AREA_DEG2 * (np.pi/180)**2

print(f"Using DES survey area: {DES_SURVEY_AREA_DEG2} deg² ({DES_SURVEY_AREA_SR:.6f} sr)")

Using DES survey area: 1603.5 deg² (0.488454 sr)


## Setup and Imports

In [10]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table
import pandas as pd
import sys
import os
import configparser
import copy

# Add modules to path
sys.path.append('../../modules')
from simulation import UniverseSimulator

plt.style.use('default')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

## Load DES Y1 Data

In [11]:
# Load DES Y1 RedMaPPer catalogue
file_path = '/sps/euclid/Users/cmurray/clusters_likelihood/redmapper_y1a1_public_v6.4_catalog.fits.gz'
redmapper_catalogue = Table.read(file_path, format='fits')

# Clean catalogue
redmapper_catalogue.remove_columns(['P_CEN', 'RA_CEN', 'DEC_CEN', 'ID_CEN', 'PZBINS', 'PZ'])
redmapper_catalogue = redmapper_catalogue.to_pandas()

print(f"Loaded {len(redmapper_catalogue)} clusters")
print(f"Richness range: {redmapper_catalogue['LAMBDA'].min():.1f} - {redmapper_catalogue['LAMBDA'].max():.1f}")
print(f"Redshift range: {redmapper_catalogue['Z_LAMBDA'].min():.3f} - {redmapper_catalogue['Z_LAMBDA'].max():.3f}")

Loaded 6729 clusters
Richness range: 20.0 - 234.5
Redshift range: 0.200 - 0.858


## Define Binning Scheme (Excluding Final Richness Bin)

In [12]:
# DES binning - exclude final richness bin (>60)
des_lambda_bins = np.array([20, 30, 45, 60])  # Exclude 240 bin
des_z_bins = np.array([0.2, 0.35, 0.5, 0.65])

n_lambda_bins = len(des_lambda_bins) - 1
n_z_bins = len(des_z_bins) - 1
n_total_bins = n_lambda_bins * n_z_bins

print(f"Richness bins: {des_lambda_bins}")
print(f"Redshift bins: {des_z_bins}")
print(f"Total bins for calibration: {n_total_bins} ({n_lambda_bins} λ × {n_z_bins} z)")

# Bin centers for plotting
lambda_centers = 0.5 * (des_lambda_bins[:-1] + des_lambda_bins[1:])
z_centers = 0.5 * (des_z_bins[:-1] + des_z_bins[1:])

print(f"Richness bin centers: {lambda_centers}")
print(f"Redshift bin centers: {z_centers}")

Richness bins: [20 30 45 60]
Redshift bins: [0.2  0.35 0.5  0.65]
Total bins for calibration: 9 (3 λ × 3 z)
Richness bin centers: [25.  37.5 52.5]
Redshift bin centers: [0.275 0.425 0.575]


## Extract DES Observed Data Vector

In [13]:
# Extract observed number counts in each redshift bin
def get_des_counts_by_z_bin(catalogue, lambda_bins, z_bins):
    """Extract number counts in richness-redshift bins."""
    counts = []
    
    for i in range(len(z_bins) - 1):
        z_min, z_max = z_bins[i], z_bins[i+1]
        z_mask = (catalogue['Z_LAMBDA'] >= z_min) & (catalogue['Z_LAMBDA'] < z_max)
        
        n_clusters, _ = np.histogram(catalogue['LAMBDA'][z_mask], bins=lambda_bins)
        counts.append(n_clusters)
        
        print(f"z-bin {z_min:.2f}-{z_max:.2f}: {n_clusters} (total: {n_clusters.sum()})")
    
    return np.array(counts)

# Get observed counts
des_observed_counts = get_des_counts_by_z_bin(redmapper_catalogue, des_lambda_bins, des_z_bins)
print(f"\nTotal observed clusters in calibration bins: {des_observed_counts.sum()}")
print(f"Observed counts shape: {des_observed_counts.shape} (z_bins × λ_bins)")

z-bin 0.20-0.35: [762 376 123] (total: 1261)
z-bin 0.35-0.50: [1549  672  187] (total: 2408)
z-bin 0.50-0.65: [1612  687  205] (total: 2504)

Total observed clusters in calibration bins: 6173
Observed counts shape: (3, 3) (z_bins × λ_bins)


In [14]:
# DES Y1 published mean lensing masses (Table II, with h correction)
# Note: Excluding final richness bin, so only first 3 richness bins
des_mean_masses = {
    'z_bin_0': np.array([14.036, 14.323, 14.454]) + np.log10(0.744),  # z: 0.2-0.35
    'z_bin_1': np.array([14.007, 14.291, 14.488]) + np.log10(0.744),  # z: 0.35-0.5  
    'z_bin_2': np.array([13.929, 14.301, 14.493]) + np.log10(0.744)   # z: 0.5-0.65
}

# Uncertainties (stat + sys)
des_mass_errors = {
    'z_bin_0': np.array([0.032 + 0.045, 0.031 + 0.051, 0.044 + 0.050]),
    'z_bin_1': np.array([0.033 + 0.056, 0.031 + 0.061, 0.044 + 0.065]),
    'z_bin_2': np.array([0.048 + 0.072, 0.041 + 0.086, 0.056 + 0.068])
}

# Organize as 2D arrays for easy comparison
des_observed_masses = np.array([des_mean_masses['z_bin_0'], 
                               des_mean_masses['z_bin_1'], 
                               des_mean_masses['z_bin_2']])

des_mass_uncertainties = np.array([des_mass_errors['z_bin_0'],
                                  des_mass_errors['z_bin_1'], 
                                  des_mass_errors['z_bin_2']])

print(f"DES mean masses shape: {des_observed_masses.shape}")
print(f"Mean mass range: {des_observed_masses.min():.3f} - {des_observed_masses.max():.3f}")

DES mean masses shape: (3, 3)
Mean mass range: 13.801 - 14.365


## Setup Capish Simulation

In [15]:
def create_capish_config(mass_richness_params, cosmo_params, survey_area_sr):
    """Create Capish configuration with given parameters."""
    
    config = configparser.ConfigParser()
    
    # Parameters section (unified)
    config['parameters'] = {**cosmo_params, **mass_richness_params}
    
    # Halo catalogue settings
    config['halo_catalogue'] = {
        'hmf_name': 'Despali16',
        'bias_model': 'Tinker10',
        'recompute_SSC_ficucial': 'False',
        'name_sigma2ij_fullsky_file': 'sigma2ij_fullsky_fiducial_zmin={}_zmax={}_nzbins={}.pkl',
        'save_new_SSC_fiducial': 'False',
        'mass_def_overdensity_type': 'critical',
        'mass_def_overdensity_delta': 'vir',
        'Mstar': '10',
        's': '0.0',
        'q': '1.0',
        'sky_area': str(survey_area_sr),
        'log10m_min': '13.5',
        'log10m_max': '16.0',
        'n_mass_bins': '1000',
        'z_min': '0.1',
        'z_max': '0.8',
        'n_redshift_bins': '700',
        'SSC': 'False',
        'Omega_c_fiducial': '0.25',
        'Omega_b_fiducial': '0.049',
        'h_fiducial': '0.674',
        'sigma_8_fiducial': '0.811',
        'n_s_fiducial': '0.965'
    }
    
    # Cluster catalogue settings
    config['cluster_catalogue'] = {
        'add_completeness': 'False',  # Simplify for initial calibration
        'params_completeness': '1.0, 0.0, 14.0, 0.0',
        'add_photometric_redshift': 'False',
        'add_purity': 'False',
        'params_purity': '1.0, 0.0, 2.0, 0.0',
        'add_selection': 'False'
    }
    
    # Mass-observable relation
    config['cluster_catalogue.mass_observable_relation'] = {
        'which_relation': 'Gauss+Poiss-corr'
    }
    
    # Photometric redshift (not used)
    config['cluster_catalogue.photometric_redshift'] = {
        'sigma_z0': '0.02'
    }
    
    # Summary statistics matching DES binning
    config['summary_statistics'] = {
        'summary_statistic': 'binned_count_mean_mass',
        'Gamma': '0.7',
        'richness_lower_limit': '20',
        'richness_upper_limit': '60',  # Exclude final bin
        'n_bins_richness': str(n_lambda_bins),
        'richness_edges': ', '.join(map(str, des_lambda_bins)),
        'redshift_lower_limit': '0.2',
        'redshift_upper_limit': '0.65',
        'n_bins_redshift': str(n_z_bins),
        'redshift_edges': ', '.join(map(str, des_z_bins)),
        'log10mWL_lower_limit': '13',
        'log10mWL_upper_limit': '16',
        'n_bins_log10mWL': '5',
        'log10mWL_edges': '12, 13.2, 13.8, 14, 14.8, 17'
    }
    
    return config

# Create initial configuration
capish_config = create_capish_config(MASS_RICHNESS_PARAMS, COSMO_PARAMS, DES_SURVEY_AREA_SR)
print("Capish configuration created")
print(f"Survey area: {capish_config['halo_catalogue']['sky_area']} sr")

Capish configuration created
Survey area: 0.4884540326279872 sr


## Run Capish Simulation

In [16]:
def run_capish_prediction(config, variable_params=['Omega_m', 'sigma8']):
    """Run Capish simulation and return binned predictions."""
    
    # Initialize simulator
    simulator = UniverseSimulator(
        default_config=config,
        variable_params_names=variable_params
    )
    
    # Use fiducial cosmology (parameters already in config)
    fiducial_values = [float(config['parameters']['Omega_m']), 
                      float(config['parameters']['sigma8'])]
    
    print(f"Running simulation with Ωm={fiducial_values[0]}, σ8={fiducial_values[1]}")
    
    # Run simulation
    summary_stats = simulator.run_simulation(fiducial_values)
    
    return summary_stats

# Run the simulation
print("Running Capish simulation...")
capish_results = run_capish_prediction(capish_config)

print(f"Simulation complete")
print(f"Results type: {type(capish_results)}")
if hasattr(capish_results, 'shape'):
    print(f"Results shape: {capish_results.shape}")
elif isinstance(capish_results, (list, tuple)):
    print(f"Results length: {len(capish_results)}")
    if len(capish_results) > 0:
        print(f"First element type: {type(capish_results[0])}")
        if hasattr(capish_results[0], 'shape'):
            print(f"First element shape: {capish_results[0].shape}")

Running Capish simulation...
Running simulation with Ωm=0.315, σ8=0.811


ModuleNotFoundError: No module named 'camb'

## Process Capish Results

In [ ]:
# Extract counts and masses from Capish results
# Note: This depends on the actual format returned by the summary statistics

def extract_predictions(results, n_z_bins, n_lambda_bins):
    """Extract number counts and mean masses from Capish results."""
    
    if isinstance(results, tuple) and len(results) == 2:
        # Format: (counts, mean_masses)
        counts_flat, masses_flat = results
        
        # Reshape to (n_z_bins, n_lambda_bins)
        predicted_counts = counts_flat.reshape(n_z_bins, n_lambda_bins)
        predicted_masses = masses_flat.reshape(n_z_bins, n_lambda_bins)
        
    elif hasattr(results, 'shape') and len(results.shape) == 1:
        # Flat array - assume first half is counts, second half is masses
        n_total = n_z_bins * n_lambda_bins
        if len(results) == 2 * n_total:
            predicted_counts = results[:n_total].reshape(n_z_bins, n_lambda_bins)
            predicted_masses = results[n_total:].reshape(n_z_bins, n_lambda_bins)
        else:
            # Only counts provided
            predicted_counts = results.reshape(n_z_bins, n_lambda_bins)
            predicted_masses = np.zeros_like(predicted_counts)
            print("Warning: No mass predictions found, using zeros")
    
    else:
        print(f"Unexpected results format: {type(results)}")
        # Create dummy predictions for debugging
        predicted_counts = np.ones((n_z_bins, n_lambda_bins)) * 100
        predicted_masses = np.ones((n_z_bins, n_lambda_bins)) * 14.2
        print("Using dummy predictions for debugging")
    
    return predicted_counts, predicted_masses

# Extract predictions
predicted_counts, predicted_masses = extract_predictions(capish_results, n_z_bins, n_lambda_bins)

print(f"Predicted counts shape: {predicted_counts.shape}")
print(f"Predicted masses shape: {predicted_masses.shape}")
print(f"Total predicted clusters: {predicted_counts.sum():.1f}")
print(f"Mean predicted mass: {predicted_masses.mean():.3f} (where > 0)")

## Compare Data vs Theory

In [ ]:
# Create comparison plots
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Plot 1: Number counts comparison
ax1 = axes[0, 0]
x_pos = np.arange(n_total_bins)
bin_labels = [f"λ{i}_z{j}" for j in range(n_z_bins) for i in range(n_lambda_bins)]

obs_flat = des_observed_counts.flatten()
pred_flat = predicted_counts.flatten()

width = 0.35
ax1.bar(x_pos - width/2, obs_flat, width, label='DES Observed', alpha=0.7, color='blue')
ax1.bar(x_pos + width/2, pred_flat, width, label='Capish Predicted', alpha=0.7, color='red')
ax1.set_xlabel('Bin (λ_richness_z_redshift)')
ax1.set_ylabel('Number of Clusters')
ax1.set_title('Number Counts: Observed vs Predicted')
ax1.set_xticks(x_pos)
ax1.set_xticklabels(bin_labels, rotation=45, ha='right')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Mean masses comparison
ax2 = axes[0, 1]
obs_mass_flat = des_observed_masses.flatten()
obs_err_flat = des_mass_uncertainties.flatten()
pred_mass_flat = predicted_masses.flatten()

ax2.errorbar(x_pos - width/2, obs_mass_flat, yerr=obs_err_flat, 
            fmt='bo', label='DES Observed', alpha=0.7, capsize=3)
ax2.plot(x_pos + width/2, pred_mass_flat, 'rs', label='Capish Predicted', alpha=0.7, markersize=6)
ax2.set_xlabel('Bin (λ_richness_z_redshift)')
ax2.set_ylabel('log₁₀(M_WL / M_☉)')
ax2.set_title('Mean Masses: Observed vs Predicted')
ax2.set_xticks(x_pos)
ax2.set_xticklabels(bin_labels, rotation=45, ha='right')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Plot 3: Counts scatter plot
ax3 = axes[1, 0]
ax3.scatter(obs_flat, pred_flat, alpha=0.7, s=60)
max_count = max(obs_flat.max(), pred_flat.max())
ax3.plot([0, max_count], [0, max_count], 'k--', alpha=0.5, label='1:1 line')
ax3.set_xlabel('DES Observed Counts')
ax3.set_ylabel('Capish Predicted Counts')
ax3.set_title('Number Counts Correlation')
ax3.legend()
ax3.grid(True, alpha=0.3)

# Add correlation coefficient
corr_counts = np.corrcoef(obs_flat, pred_flat)[0, 1]
ax3.text(0.05, 0.95, f'r = {corr_counts:.3f}', transform=ax3.transAxes, 
         bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

# Plot 4: Mass scatter plot  
ax4 = axes[1, 1]
ax4.errorbar(obs_mass_flat, pred_mass_flat, xerr=obs_err_flat, 
            fmt='o', alpha=0.7, capsize=3)
mass_range = [obs_mass_flat.min() - 0.1, obs_mass_flat.max() + 0.1]
ax4.plot(mass_range, mass_range, 'k--', alpha=0.5, label='1:1 line')
ax4.set_xlabel('DES Observed log₁₀(M_WL)')
ax4.set_ylabel('Capish Predicted log₁₀(M_WL)')
ax4.set_title('Mean Mass Correlation')
ax4.legend()
ax4.grid(True, alpha=0.3)

# Add correlation coefficient
corr_mass = np.corrcoef(obs_mass_flat, pred_mass_flat)[0, 1]
ax4.text(0.05, 0.95, f'r = {corr_mass:.3f}', transform=ax4.transAxes,
         bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

plt.tight_layout()
plt.show()

## Calculate Goodness of Fit

In [ ]:
# Calculate chi-squared statistics
def calculate_chi_squared(observed, predicted, errors=None, data_type='counts'):
    """Calculate chi-squared statistic."""
    
    obs_flat = observed.flatten()
    pred_flat = predicted.flatten()
    
    if data_type == 'counts':
        # For Poisson counts, use sqrt(N) as uncertainty
        sigma = np.sqrt(np.maximum(obs_flat, 1))  # Avoid zero
    else:
        # For masses, use provided errors
        if errors is not None:
            sigma = errors.flatten()
        else:
            sigma = np.ones_like(obs_flat) * 0.1  # Default 10% error
    
    chi2 = np.sum((obs_flat - pred_flat)**2 / sigma**2)
    ndof = len(obs_flat)
    chi2_reduced = chi2 / ndof
    
    return chi2, ndof, chi2_reduced

# Calculate chi-squared for counts and masses
chi2_counts, ndof_counts, chi2_red_counts = calculate_chi_squared(
    des_observed_counts, predicted_counts, data_type='counts')

chi2_masses, ndof_masses, chi2_red_masses = calculate_chi_squared(
    des_observed_masses, predicted_masses, des_mass_uncertainties, data_type='masses')

# Combined chi-squared
chi2_total = chi2_counts + chi2_masses
ndof_total = ndof_counts + ndof_masses
chi2_red_total = chi2_total / ndof_total

print("=" * 60)
print("GOODNESS OF FIT SUMMARY")
print("=" * 60)
print(f"Number Counts:")
print(f"  χ² = {chi2_counts:.2f} (ndof = {ndof_counts})")
print(f"  χ²/ndof = {chi2_red_counts:.3f}")
print(f"\nMean Masses:")
print(f"  χ² = {chi2_masses:.2f} (ndof = {ndof_masses})")
print(f"  χ²/ndof = {chi2_red_masses:.3f}")
print(f"\nCombined:")
print(f"  χ²_total = {chi2_total:.2f} (ndof = {ndof_total})")
print(f"  χ²/ndof = {chi2_red_total:.3f}")

# Additional metrics
count_ratio_mean = (predicted_counts / np.maximum(des_observed_counts, 1)).mean()
count_ratio_std = (predicted_counts / np.maximum(des_observed_counts, 1)).std()
mass_diff_mean = (predicted_masses - des_observed_masses).mean()
mass_diff_std = (predicted_masses - des_observed_masses).std()

print(f"\nAdditional Metrics:")
print(f"Count ratio (pred/obs): {count_ratio_mean:.3f} ± {count_ratio_std:.3f}")
print(f"Mass difference (pred-obs): {mass_diff_mean:.3f} ± {mass_diff_std:.3f}")

# Parameter recommendations
print(f"\n" + "=" * 60)
print("PARAMETER ADJUSTMENT RECOMMENDATIONS")
print("=" * 60)
if chi2_red_total > 2.0:
    print("⚠️  Poor fit (χ²/ndof > 2)")
    if count_ratio_mean > 1.2:
        print("→ Too many predicted clusters: decrease mass-richness normalization")
        print("→ Try reducing 'params_mean_obs_mu0' or 'params_mean_obs_mulog10m'")
    elif count_ratio_mean < 0.8:
        print("→ Too few predicted clusters: increase mass-richness normalization")
        print("→ Try increasing 'params_mean_obs_mu0' or 'params_mean_obs_mulog10m'")
    
    if abs(mass_diff_mean) > 0.1:
        print(f"→ Mass offset: adjust WL mass relation parameters")
        print(f"→ Try adjusting 'params_mean_log10mWL_aWL' or 'params_mean_log10mWL_bWL'")

elif chi2_red_total < 0.5:
    print("📊 Potentially over-fitted (χ²/ndof < 0.5)")
    print("→ Consider increasing scatter parameters")
else:
    print("✅ Reasonable fit (0.5 < χ²/ndof < 2.0)")
    print("→ Fine-tune parameters for optimal agreement")

## Current Parameter Values

In [ ]:
# Display current parameters for easy reference
print("=" * 60)
print("CURRENT PARAMETER VALUES")
print("=" * 60)
print("Mass-Richness Relation:")
for key, value in MASS_RICHNESS_PARAMS.items():
    print(f"  {key:<25} = {value}")

print(f"\nCosmology:")
for key, value in COSMO_PARAMS.items():
    print(f"  {key:<10} = {value}")
    
print(f"\nSurvey:")
print(f"  Area = {DES_SURVEY_AREA_DEG2} deg² ({DES_SURVEY_AREA_SR:.6f} sr)")
print(f"  Richness bins: {des_lambda_bins}")
print(f"  Redshift bins: {des_z_bins}")

## Quick Re-run Function

In [ ]:
def quick_rerun(new_mass_richness_params=None, new_cosmo_params=None, plot=True):
    """Quickly re-run simulation with updated parameters."""
    
    # Use updated parameters or defaults
    mr_params = new_mass_richness_params if new_mass_richness_params else MASS_RICHNESS_PARAMS
    cosmo_params = new_cosmo_params if new_cosmo_params else COSMO_PARAMS
    
    # Create new config
    config = create_capish_config(mr_params, cosmo_params, DES_SURVEY_AREA_SR)
    
    # Run simulation
    results = run_capish_prediction(config)
    pred_counts, pred_masses = extract_predictions(results, n_z_bins, n_lambda_bins)
    
    # Calculate fit quality
    chi2_c, _, chi2_red_c = calculate_chi_squared(des_observed_counts, pred_counts, data_type='counts')
    chi2_m, _, chi2_red_m = calculate_chi_squared(des_observed_masses, pred_masses, des_mass_uncertainties, data_type='masses')
    chi2_total = chi2_c + chi2_m
    chi2_red_total = chi2_total / (n_total_bins * 2)
    
    print(f"Quick results: χ²/ndof = {chi2_red_total:.3f} (counts: {chi2_red_c:.3f}, masses: {chi2_red_m:.3f})")
    
    if plot:
        # Quick comparison plot
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
        
        # Counts
        obs_flat = des_observed_counts.flatten()
        pred_flat = pred_counts.flatten()
        ax1.scatter(obs_flat, pred_flat, alpha=0.7)
        max_c = max(obs_flat.max(), pred_flat.max())
        ax1.plot([0, max_c], [0, max_c], 'k--', alpha=0.5)
        ax1.set_xlabel('Observed Counts')
        ax1.set_ylabel('Predicted Counts')
        ax1.set_title(f'Counts (χ²/ndof = {chi2_red_c:.2f})')
        ax1.grid(True, alpha=0.3)
        
        # Masses
        obs_mass_flat = des_observed_masses.flatten()
        pred_mass_flat = pred_masses.flatten()
        obs_err_flat = des_mass_uncertainties.flatten()
        ax2.errorbar(obs_mass_flat, pred_mass_flat, xerr=obs_err_flat, fmt='o', alpha=0.7, capsize=3)
        mass_range = [obs_mass_flat.min()-0.1, obs_mass_flat.max()+0.1]
        ax2.plot(mass_range, mass_range, 'k--', alpha=0.5)
        ax2.set_xlabel('Observed Mass')
        ax2.set_ylabel('Predicted Mass')
        ax2.set_title(f'Masses (χ²/ndof = {chi2_red_m:.2f})')
        ax2.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
    
    return pred_counts, pred_masses, chi2_red_total

# Example usage (uncomment and modify parameters as needed):
# new_params = MASS_RICHNESS_PARAMS.copy()
# new_params['params_mean_obs_mu0'] = 3.0  # Reduce normalization
# quick_rerun(new_params)